In [68]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

%matplotlib qt

In [69]:
points = 1000
x = np.linspace(0.000001, 2*np.pi, points, endpoint=False).reshape(points,1)
y1 = np.sin(x)**2
y2 = np.cos(x)**2

In [70]:
def matrix(x):
    matrics = np.zeros([len(x), len(x)])
    for i in range(len(x)):
        for j in range(len(x)):
            matrics[i][j] = np.abs(np.sin(x[i] - x[j]))
    return matrics

In [71]:
matrix = matrix(x)
print (matrix)

[[0.         0.00628314 0.01256604 ... 0.01884944 0.01256704 0.00628414]
 [0.00628314 0.         0.00628314 ... 0.02513109 0.01884944 0.01256704]
 [0.01256604 0.00628314 0.         ... 0.03141175 0.02513109 0.01884944]
 ...
 [0.01884944 0.02513109 0.03141175 ... 0.         0.00628314 0.01256604]
 [0.01256704 0.01884944 0.02513109 ... 0.00628314 0.         0.00628314]
 [0.00628414 0.01256704 0.01884944 ... 0.01256604 0.00628314 0.        ]]


In [72]:
def free_energy(yhat, x, matrix, rho, lambd):
    L = len(yhat)
    dx = x[1] - x[0]
    # first_term = (2 * np.pi / (L)) * (np.dot(yhat.T, np.log(rho * yha t)))  # 虽说前面的系数其实是有影响的， 但是单就这一项而言
    # 他只是一个系数， 影响其实没有那么大。
    first_term = integrate.simps((yhat*np.log(rho*yhat)).T, x.T)
    #try simpson rule by myself
    # first_term = (3/dx) * (yhat[0] + yhat[-1]) +
    second_term = (2 * np.pi ** 2 * rho / (L) ** 2) * np.dot(np.dot(yhat.T, matrix), yhat)
    third_term = lambd * integrate.trapz(integrate.simps(yhat.T, x.T) - 1)
    loss = first_term \
           + second_term \
           + third_term
    return first_term, second_term,third_term,loss

In [73]:
print ("this is the result of sin")
free_energy(y1,x,matrix,6,20)

this is the result of sin


(array([4.41539938]), array([[15.70787527]]), 0.0, array([[20.12327465]]))

In [74]:
print ("this is the result of cos")
free_energy(y2,x,matrix,6,20)

this is the result of cos


(array([4.40413929]), array([[15.70786727]]), 0.0, array([[20.11200656]]))

### 验证直线项

In [75]:
line1 = np.ones(len(x)) - (1-(1/(2*np.pi)))
line1 = line1.reshape(points,1)
line2 = line1.copy()
line2[0] = 0.05
line2[-1] = 0.05

# the line2 is modified result, which is the two end points are lower

integral of lines

In [76]:
integrate.trapz(line1.T, x.T)

array([0.99899984])

In [77]:
free_energy(line1, x, matrix, 4, 20)

(array([-0.45113105]), array([[1.27323536]]), 0.0, array([[0.82210431]]))

In [79]:
free_energy(line2, x, matrix, 4, 20)

(array([-0.45117608]), array([[1.26974242]]), 0.0, array([[0.81856634]]))

第一项的解决方法
1. 第一项和积分的值有关，积分越接近1， 误差越小
2. 有时， 两个端点的问题，会造成熵为负值，但是香农熵其实始终是正直，所以我们可以加上abs来搞

第二项的解决方法：



In [51]:
np.dot(np.dot(line1.T, matrix), line1)

array([[16093.51854502]])

In [52]:
np.dot(np.dot(line2.T, matrix), line2)

array([[16049.32413103]])

In [53]:
#试着自己写一个simpson rule
test_y = np.sin(x)**2
# plt.plot(x,test_y)

In [54]:
# simpson matrix
dx = x[1] - x[0]
simpson_matrix = np.zeros(len(x))
for i in range (len(x)):
    if i%2 == 0:
        simpson_matrix[i] = 2
    if i%2 != 0:
        simpson_matrix[i] = 4
simpson_matrix[0], simpson_matrix[-1] = 1, 1

In [55]:
I = (dx/3) * np.dot(simpson_matrix, test_y)
print (I)   # x 最好有奇数个

[3.14159257]


In [56]:
def first_term(yhat, simpson_matrix, rho):
    I = (dx/3) * np.dot(simpson_matrix,yhat * np.log(rho * yhat))
    return I

In [57]:
first_term(line1,simpson_matrix, 4)

array([-0.4511306])